# Числа Фибоначчи

Познакомимся с рекурсией на примере чисел Фибоначчи.
Это последовательность, которая как раз определена рекурсивно - каждое следующее число равно сумме двух предыдущих.
Первые 2 числа - 1, 1 (иногда 0, 1).

1, 1, 2, 3, 5, 8, 13, 21, 34 ...

### рекурсивная функция

In [1]:
# Рекурсивная функция - это такая функция, которая вызывает саму себя.
# Данная функция позволяет вычислить n-ное число Фибоначчи по определению.
def fib(n):
    if n <= 1:
        return 1
    
    return fib(n - 1) + fib(n - 2)

In [2]:
fib(20)

10946

In [3]:
%%time
fib(30)

CPU times: user 164 ms, sys: 1.73 ms, total: 166 ms
Wall time: 165 ms


1346269

In [4]:
%%time
fib(36)

CPU times: user 2.88 s, sys: 21.9 ms, total: 2.9 s
Wall time: 2.9 s


24157817

У рекурсивного решения этой задачи есть существенный недостаток - из-за того, что каждый раз функция вызывает 2 копии себя, мы получаем экспоненциальную сложность порядка **O(2^n)**. Это видно на примерах выше: для **n=30** функция работает 222ms, а для **n=35** - уже в 10 раз дольше. Для **n=100** мы вообще не дождемся её завершения.

Формально оценить сложность рекурсивной функции можно при помощи рекуррентного соотношения

$
T(n) = T(n - 1) + T(n - 2) + C \\
T(n) \le 2T(n - 1) + C \\
T(n) \le 4T(n - 2) + (2 + 1) C \\
T(n) \le 2^k(n - k) + (2^k - 1) C  \\
T(n) \le 2^n * C1 \\
$

### мемоизация при помощи lru_cache

Проблема рекурсивного решения в том, что мы много раз вызываем функцию с одними и теми же аргументами. 
Для борьбы с этой проблемой применяют мемоизацию - запоминание результата работы функции с заданным набором аргументов.

In [5]:
# воспользуемся декоратором lru_cache, который как раз реализует мемоизацию 
# он запоминает результат вызова функции для maxsize последних аргументов
from functools import lru_cache

@lru_cache(maxsize=1000)
def fib_lru(n):
    if n <= 1:
        return n
    return fib_lru(n - 1) + fib_lru(n - 2)

In [6]:
def fib_memo(n, memo: dict):
    if n in memo:
        return memo[n]
    if n <= 1:
        return n
    
    fib_curr =  fib_lru(n - 1) + fib_lru(n - 2)
    memo[n] = fib_curr
    return fib_curr

In [7]:
%%time
memo = {}
fib_memo(500, memo)

CPU times: user 660 µs, sys: 106 µs, total: 766 µs
Wall time: 778 µs


139423224561697880139724382870407283950070256587697307264108962948325571622863290691557658876222521294125

С использованием мемоизации мы понизили сложность по времени до линейной. Теперь функция вызывается ровно 1 раз для каждого значения n.
Но, решения с мемоизацией всегда требуют дополнительную память, чтобы хранить результаты.

### итеративное решение

Любую рекурсивную функцию можно переписать с использованием цикла. Для нашей задачи это сделать достаточно просто.

In [8]:
def fib_no_recusrion(n: int):
    fib0, fib1 = 0, 1
    for _ in range(n):
        fib0, fib1 = fib1, fib0 + fib1
        
    return fib0
        
        

In [9]:
%%time
x = fib_no_recusrion(1000000)

CPU times: user 11.7 s, sys: 62.1 ms, total: 11.8 s
Wall time: 11.8 s


Это решение также линейно, но не требует дополнительной памяти.

# Обход словаря

Представьте, что вы получили данные с неизвестной структурой в формате json. 
Это один из самых распространенных форматов обмена данными, по структуре он почти идентичен контейнеру dict в Python.

In [10]:
# Для работы с json в стандартной библиотеке Python есть одноименный модуль.
# Но есть сторонние библиотеки, которые позволяют делать это более эффективно, например orjson

import json
import orjson

with open('example.json', 'r') as f:
    json_str = f.read()
   

FileNotFoundError: [Errno 2] No such file or directory: 'example.json'

In [ ]:
%%timeit
data = json.loads(json_str)

In [ ]:
%%timeit
data = orjson.loads(json_str)

In [ ]:
data = orjson.loads(json_str)

Напишем рекурсивную функцию обхода: для ключей типа dict будем вызывать саму себя,
а для ключей типа list будем вызывать обход для первого элемента.

In [ ]:
# аргумент level нужен нам, чтобы определять уровень вложенности для красивой печати

def traverse(level: int, obj: dict):
    if isinstance(obj, dict):
        for key, value in obj.items():
            print('  ' * level + '|__', key)
            traverse(level + 1, value)
            
    if isinstance(obj, list):
        print('  ' * level + '|__', '[]')
        traverse(level + 1, obj[0])
        
    return
            

In [ ]:
traverse(0, data)

Теперь мы видим структуру наших данных и можем быстро достать то, что нужно.

In [ ]:
data['result']['list'][0]['value']['list'][0]['value']['service_item']['title']

# Хвостовая рекурсия

Хвостовая рекурсия - это особый вид рекурсии, где последней инструкцией выполняется рекурсивный вызов самой функции.


Рассмотрим рекурсивную функцию вычисления факториала при помощи модуля dis. 
Он позволяет смотреть, какие инструкции выполнит интерпретатор при вызове функции.

In [ ]:
from dis import dis

In [ ]:
def factorial(n: int):
    if n == 0:
        return 1

    return n * factorial(n - 1)

Видно, что на шаге 22 вызывается инструктция **CALL_FUNCTION**, а после неё идет умножение **BINARY_MULTIPLY**.

Изменим функцию, чтобы она удовлетворяла свойству хвостовой рекурсии. Здесь нам понадобится аккумулятор.

In [ ]:
def factorial_tail(n: int, acc: int = 1):
    if n == 1:
        return acc

    return factorial_tail(n - 1, acc * n)

In [ ]:
dis(factorial_tail)

Здесь инструкция **CALL_FUNCTION** идет в конце.

Преимущество хвостовой рекурсии в том, что ей не нужно хранить весь стек вызовов, чтобы вернуть результат. 
Он уже будет накоплен в аккумуляторе и мы можем не подниматься обратно по цепочке вызовов, в отличие от обычной рекурсии.

В некоторых языках программирования (в основном, фукнциональных), эта оптимизация происходит на этапе компиляции, 
и поэтому хвостовая рекурсия не уступает в производительности циклам.
В Python, это, к сожалению, не так, но при помощи манипуляций с байткодом можно это реализовать.
https://towardsdatascience.com/python-stack-frames-and-tail-call-optimization-4d0ea55b0542

# Множество всех подмножеств 

Рекурсия может быть полезна для решения "переборных" задач, например, для получения всех комбинаций.

Определим базовый случай: множество подмножеств пустого множества состоит из пустого множества.

Шаг рекурсии: если мы имеем все подмножества множества из **N-1** элементов, то, добавив к кажому из них еще один элемент, мы получим решение для **N**.

In [ ]:
def get_subsets(original_set: list):
    if len(original_set) == 0:
        return [[]]
    
    all_subsets = [[]]
    for i, elem in enumerate(original_set):
        all_subsets += [[elem] + subset for subset in get_subsets(original_set[i + 1:])]
        
    return all_subsets
    
    
    

    
    

In [ ]:
get_subsets([1, 2, 3])